# Finding Missing Person

### Importing Neccessary Libraries

In [29]:
pwd

'/home/wsuser/work'

In [30]:
!pip install keras==2.2.4
!pip install tensorflow

In [31]:
import numpy as np#used for numerical analysis
#import keras
import tensorflow #open source used for both ML and DL for computation
from tensorflow.keras.models import Sequential #it is a plain stack of layers
from tensorflow.keras import layers #A layer consists of a tensor-in tensor-out computation function
#Dense layer is the regular deeply connected neural network layer
from tensorflow.keras.layers import Dense,Flatten
#Faltten-used fot flattening the input or change the dimension
from tensorflow.keras.layers import Conv2D,MaxPooling2D #Convolutional layer
#MaxPooling2D-for downsampling the image
from keras.preprocessing.image import ImageDataGenerator


In [32]:
tensorflow.__version__

'2.4.4'

In [33]:
tensorflow.keras.__version__

'2.4.0'

### Image Data Agumentation

In [34]:
#setting parameter for Image Data agumentation to the training data
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
#Image Data agumentation to the testing data
test_datagen=ImageDataGenerator(rescale=1./255)

### Loading our data and performing data agumentation

In [35]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_0a0a97d030cb478d8237e5fc840d4cd4 = 'https://s3.us.cloud-object-storage.appdomain.cloud'
else:
    endpoint_0a0a97d030cb478d8237e5fc840d4cd4 = 'https://s3.private.us.cloud-object-storage.appdomain.cloud'

client_0a0a97d030cb478d8237e5fc840d4cd4 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='AMI91n78AoECDRxw9SnKWTXuGbt2vnfs-x74u3IE3aAI',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_0a0a97d030cb478d8237e5fc840d4cd4)

streaming_body_1 = client_0a0a97d030cb478d8237e5fc840d4cd4.get_object(Bucket='missingpersons-donotdelete-pr-ad7fyi7owyqutr', Key='dataset.zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [36]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [37]:
pwd

'/home/wsuser/work'

In [38]:
import os
filenames = os.listdir('/home/wsuser/work/dataset/trainset')

In [39]:
#performing data agumentation to train data
x_train = train_datagen.flow_from_directory('/home/wsuser/work/dataset/trainset',target_size=(64, 64),batch_size=5,
                                            color_mode='rgb',class_mode='binary')
#performing data agumentation to test data
x_test = test_datagen.flow_from_directory('/home/wsuser/work/dataset/testset',target_size=(64, 64),batch_size=5,
                                            color_mode='rgb',class_mode='binary') 

Found 174 images belonging to 2 classes.
Found 60 images belonging to 2 classes.


In [40]:
print(x_train.class_indices)#checking the number of classes

{'Found Missing': 0, 'Normal': 1}


In [41]:
print(x_test.class_indices)#checking the number of classes

{'Found Missing': 0, 'Normal': 1}


In [42]:
from collections import Counter as c
c(x_train .labels)

Counter({0: 80, 1: 94})

### Creating the model

In [43]:
# Initializing the CNN
classifier = Sequential()

# First convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# Second convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening the layers
classifier.add(Flatten())

# Adding a fully connected layer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=1, activation='sigmoid')) # softmax for more than 2



In [44]:
classifier.summary()#summary of our model

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               802944    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                

### Compiling the model

In [45]:
# Compiling the CNN
# categorical_crossentropy for more than 2
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) 

## Fitting the model

In [46]:
classifier.fit_generator(
        generator=x_train,steps_per_epoch = len(x_train),
        epochs=20, validation_data=x_test,validation_steps = len(x_test))# No of images in test set

/opt/conda/envs/Python-3.8-main/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1839: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
35/35 [==============================] - 4s 86ms/step - loss: 0.7165 - accuracy: 0.5571 - val_loss: 0.6938 - val_accuracy: 0.3833
Epoch 2/20
35/35 [==============================] - 3s 76ms/step - loss: 0.6911 - accuracy: 0.5082 - val_loss: 0.6971 - val_accuracy: 0.6667
Epoch 3/20
35/35 [==============================] - 3s 77ms/step - loss: 0.6523 - accuracy: 0.5721 - val_loss: 0.7037 - val_accuracy: 0.4667
Epoch 4/20
35/35 [==============================] - 3s 77ms/step - loss: 0.6854 - accuracy: 0.6077 - val_loss: 0.7532 - val_accuracy: 0.6500
Epoch 5/20
35/35 [==============================] - 3s 80ms/step - loss: 0.6603 - accuracy: 0.6271 - val_loss: 0.7827 - val_accuracy: 0.3167
Epoch 6/20
35/35 [==============================] - 3s 78ms/step - loss: 0.6453 - accuracy: 0.6327 - val_loss: 0.7745 - val_accuracy: 0.4167
Epoch 7/20
35/35 [==============================] - 3s 81ms/step - loss: 0.6286 - accuracy: 0.6863 - val_loss: 0.7825 - val_accuracy: 0.4333
Epoch 8/20
35

### Saving our model

In [47]:
# Save the model
classifier.save('missing_person.h5')

In [48]:
!tar -zcvf detecting-missing-persons_new.tgz missing_person.h5

missing_person.h5


In [49]:
ls -1

dataset/
detecting-missing-persons_new.tgz
missing_person.h5
model-bw.json


In [50]:
model_json = classifier.to_json()
with open("model-bw.json", "w") as json_file:
    json_file.write(model_json)

In [51]:
!pip install watson-machine-learning-client --upgrade

In [52]:
# Replace the credentials that you got from Watson Machine learning service
from ibm_watson_machine_learning import APIClient
wml_credentials = {
                    "url": "https://us-south.ml.cloud.ibm.com",
                    "apikey":"66SQRoA0ymIIt4BRCL2s_gM169tys-jVBxE3uFCH3SsB"             
}
client = APIClient(wml_credentials)

In [53]:
client = APIClient(wml_credentials)

In [57]:
def guid_from_space_name(client,space_name):
    space = client.spaces.get_details()
    #print(space)
    return(next(item for item in space['resources'] if item['entity']['name']== space_name)['metadata']['id'])

In [58]:
space_uid = guid_from_space_name(client,'missingpersons')
print("Space UID= "+ space_uid)

Space UID= 5bd10b48-ee01-4d82-9692-bc9a20a04d99


In [59]:
client.set.default_space(space_uid)

'SUCCESS'

In [61]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
scikit-learn_0.22-py3.6        154010fa-5b3b-4ac1-82af-4d5ee5abbc85  base
default_r3.6                   1b70aec3-ab34-4b87-8aa0-a4a3c8296a36  base
pytorch-onnx_1.3-py3.6         1bc6029

In [62]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_1.15-py3.6")
software_spec_uid

'2b73a275-7cbf-420b-a912-eae7f436e0bc'

In [63]:
model_details = client.repository.store_model(model='detecting-missing-persons_new.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"CNN",
    client.repository.ModelMetaNames.TYPE:"keras_2.2.4",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid}
                                             )
model_id = client.repository.get_model_uid(model_details)


Note: Warnings!! :  Model type keras_2.2.4 is deprecated. We recommend you use a supported model type. See Supported Frameworks https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/pm_service_supported_frameworks.html


In [64]:
model_id

'6ce24188-6268-4cd2-9fdc-583fe57dfb3c'

In [65]:
client.repository.download(model_id,'my_model.tar.gz')

Successfully saved model content to file: 'my_model.tar.gz'


'/home/wsuser/work/my_model.tar.gz'

In [71]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [72]:
model = load_model("missing_person.h5")

### Predicting our results

In [81]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image
model = load_model("missing_person.h5") #loading the model for testing

In [82]:
img = image.load_img(r"/home/wsuser/work/dataset/trainset/Found Missing/gettyimages-1148852-612x612.jpg",grayscale=False,
                     target_size= (64,64))#loading of the image
x = image.img_to_array(img)#image to array
x = np.expand_dims(x,axis = 0)#changing the shape
#pred=model.predict_classes(x)
y=int (model.predict(x))
print(y)
#pred = np.argmax(y,axis=1)#predicting the classes
#pred

0


In [83]:
index=['Found Missing', 'Normal']
result=str(index[y])
result

'Found Missing'